# Loading libraries and modules

In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
import drivers
#reload(drivers.Agilent_EXA)

In [4]:
from importlib import reload
import lib2
from lib2.Measurement import *
from lib2.SingleToneSpectroscopy import *
from lib2 import MeasurementResult
from lib2.FourWaveMixingBase import *

# Single-Tone Spectroscopy


In [ ]:
STS = SingleToneSpectroscopy("II-qubit_search", "SPS_old_5-0.5 fF", vna = 'vna2', src = 'yok1')
vna_parameters = {"bandwidth":50, "freq_limits":(4000e6, 5000e6), "nop":401, "power":-35, "averages":1}
STS.set_fixed_parameters(vna_parameters)
STS.set_swept_parameters({'current': (STS._src.set_current, linspace(-1e-3, 1e-3, 201))})
result = STS.launch()
result.visualize()

In [ ]:
vna2 = STS._actual_devices['vna2']

In [ ]:
vna2.del_all_meas()

In [ ]:
vna2.define_Sij(4,3)

_______________

## Four Wave mixing Continuous

In [ ]:
reload(drivers.E8257D);

In [5]:
#reload(lib2.Measurement)
reload(lib2.FourWaveMixingBase)
from lib2.FourWaveMixingBase import *

In [6]:
WMM = FourWaveMixingBase('I-Measurement_with_2_gens', 'SPS_old', 60, exa = 'exa', src_plus = 'mxg', src_minus = 'exg')

The device exa is detected as EXA
The device mxg is detected as MXG
The device exg is detected as EXG
Enter your comment: 


In [7]:
exa_pars = {"bandwidth":1, "nop":2001, "centerfreq":4.2825e9, "span":100e3, "averages":50, "avg_status":1}
WMM.set_fixed_parameters(exa_pars,delta = 1e3)

In [8]:
powers = linspace(-29,9,161)
WMM.set_swept_parameters(WMM.powers_equal_setter_with_offset,powers)

In [9]:
result = WMM.launch()

Started at:  2017-05-03 19:15:31.848889


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 4 h 30 m 52.45 s, powers at $\omega_{p}$: -2.90e+01]: , average cycle time: 101.58 s       

C:\Users\PAINMASTER\Measurement\lib2\FourWaveMixingBase.py:214: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  peaks_data = [power_data[:,i] for i in self._peaks_indices]


Time left: 0 h 5 m 12.03 s, powers at $\omega_{p}$: 8.29e+00]: , average cycle time: 104.01 s         Error: power must be between -20 and 19 dBm
Time left: 0 h 3 m 28.03 s, powers at $\omega_{p}$: 8.52e+00]: , average cycle time: 104.02 s       Error: power must be between -20 and 19 dBm
Time left: 0 h 1 m 44.02 s, powers at $\omega_{p}$: 8.76e+00]: , average cycle time: 104.02 s       Error: power must be between -20 and 19 dBm
Time left: 0 h 0 m 0.0 s, powers at $\omega_{p}$: 9.00e+00]: , average cycle time: 104.02 s       

________________

In [ ]:
WMM.close_devs(['exa','mxg','exg'])

_________________

### Result actions:

In [101]:
last_data = result.get_data()

In [107]:
plt.plot(result._prepare_data_for_plot(last_data)[3])
plt.show()

C:\Users\PAINMASTER\Measurement\lib2\FourWaveMixingBase.py:204: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  XX, YY = generate_mesh(data[self._parameter_name], data["frequency"]/1e3)


In [62]:
result.get_context().get_equipment()

{'exa': {'averages': 3,
  'avg_status': 1,
  'bandwidth': 20,
  'centerfreq': 4282500000.0,
  'nop': 501,
  'span': 50000.0},
 'src_minus': {'frequency': 4282499000.0},
 'src_plus': {'frequency': 4282501000.0}}

In [ ]:
result.save()

In [10]:
result.visualize()

C:\Users\PAINMASTER\Measurement\lib2\FourWaveMixingBase.py:214: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  peaks_data = [power_data[:,i] for i in self._peaks_indices]



______________

______________

In [35]:
sys.

1013

In [32]:
x = [1,2,3,4,5,6]
yl = [[1,2,3,4,5,6],[6,5,4,3,2,1],[3,6,9,12,15,18]]
lines = [plt.plot(x,yl[i]) for i in range(len(yl))]

In [33]:
lines.pop(0)

In [35]:
plt.draw()

In [ ]:
vna2 = meas._actual_devices['vna2']
yok = meas._actual_devices['yok1']

In [ ]:
yok.set_current_limits(-12e-3,12e-3)